## Задачи на знание SQL для приглашения на стажировку

В проекте 3 задачи. В условии прописана схема данных: таблицы, названия полей, основные и внешние ключи.

Таблица 1: Покупатели (buyers) 
| Ключ |  Наименование поля| Комментарий |
|----------|----------|----------|
| PK   | buyer_id   |    |
|    | name   |    |
|     | city  |    |


 
 
   
   
 
Таблица 2: Товары (products)
| Ключ |  Наименование поля| Комментарий |
|----------|----------|----------|
| PK   | product_id  |    |
|    | product_name   |   |
|     | price   |    |
 
   
   
 
 
Таблица 3: Заказы (orders)
| Ключ |  Наименование поля| Комментарий |
|----------|----------|----------|
| PK   | order_id  |    |
|    | order_date   |   |
|   FK  | buyer_id   |    |
|   PK   | product_id   |    |
|     | quantity   |  Количество купленного товара  |


In [1]:
import pandas as pd
import duckdb

Сгенерируем набор данных с помощью большой языковой модели.
Промпт для генерации: «Привет! Сгенерируй мне, пожалуйста, правдоподобные данные: три таблицы в формате csv вот по такой схеме. Пусть уникальных покупателей будет 100, а количество их заказов и количество товаров в них - случайными числами. У каждого пользователя может быть несколько заказов, а в одном заказе может быть несколько товаров. Названия городов могут повторяться, а в самом списке этих городов должна быть Москва. Названия городов нужны только в виду самих названий, без префиксов типа "г.". Мне нужны данные за 2023, 2024 и 2025 годы.»

Загрузим получившиеся таблицы и прочитаем их.

In [16]:
buyers = pd.read_csv('buyers.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [17]:
buyers.head()

,buyer_id,name,city
0,1,Семенова Юлия Вячеславовна,Москва
1,2,Тит Чеславович Мясников,Санкт-Петербург
2,3,Харлампий Филиппович Воронов,Казань
3,4,Силин Касьян Ефимович,Екатеринбург
4,5,Федорова Октябрина Кирилловна,Новосибирск


In [18]:
orders.dtypes

order_id      object
order_date    object
buyer_id       int64
product_id     int64
quantity       int64
dtype: object

In [19]:
# поменяем тип колонки с датой на корректный
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders.dtypes

order_id              object
order_date    datetime64[ns]
buyer_id               int64
product_id             int64
quantity               int64
dtype: object

In [20]:
orders.head()

,order_id,order_date,buyer_id,product_id,quantity
0,71f4e50e-63c1-4d25-8b12-ee4c096e6f5c,2024-04-22,1,46,17
1,63651035-e594-40ef-bfac-3a49930dd807,2024-03-21,1,47,16
2,63651035-e594-40ef-bfac-3a49930dd807,2024-03-21,1,14,17
3,63651035-e594-40ef-bfac-3a49930dd807,2024-03-21,1,13,6
4,63651035-e594-40ef-bfac-3a49930dd807,2024-03-21,1,44,16


In [29]:
products.head() # отметим находчивость нейросети в части генерации названий продуктов и имен клиентов и двинемся дальше

,product_id,product_name,price
0,1,Задрать,326.99
1,2,Военный,384.22
2,3,Расстегнуть,809.27
3,4,Ставить,986.09
4,5,Белье,964.10


### Задание 1: 
Выведите имя покупателя, совершившего покупок на наибольшую сумму за все время.

In [30]:
query_1 = """
-- решение, если мы точно знаем, что такой покупатель у нас один-единственный
SELECT name
FROM buyers
WHERE buyer_id = (
    SELECT buyer_id
    FROM (
        SELECT buyer_id, SUM(price_total) AS total_sum
        FROM (
            SELECT buyer_id, quantity * price AS price_total
            FROM orders o
            LEFT JOIN products p USING (product_id)
        ) t1
        GROUP BY buyer_id
    ) t2
    ORDER BY total_sum DESC
    LIMIT 1)
"""

In [31]:
result_1 = duckdb.query(query_1).to_df()
display(result_1)

,name
0,Медведев Эрнест Бориславович


In [32]:
query_1_alt = """
-- решение для нескольких покупателей с максимальной суммой покупок
WITH max_sum_byuyer_id AS 
(
SELECT buyer_id
FROM (SELECT buyer_id, SUM(price_total) AS total_sum
      FROM (SELECT buyer_id, quantity * price AS price_total
            FROM orders o
            LEFT JOIN products p USING (product_id)) t1
       GROUP BY buyer_id) t2
WHERE total_sum = (SELECT MAX(total_sum) 
                   FROM (SELECT buyer_id, SUM(price_total) AS total_sum
                         FROM (SELECT buyer_id, quantity * price AS price_total
                               FROM orders o
                               LEFT JOIN products p USING (product_id)) t1
                         GROUP BY buyer_id) t2)
)

SELECT name
FROM buyers
WHERE buyer_id = (SELECT buyer_id FROM max_sum_byuyer_id)
"""

In [33]:
result = duckdb.query(query_1_alt).to_df()
display(result)

,name
0,Медведев Эрнест Бориславович


### Задача 2:
Выведите наименование товара, который ни разу не был куплен в 2023 году.

In [34]:
query_2 = """

SELECT product_name
FROM products
WHERE product_id NOT IN (SELECT product_id
                        FROM orders
                        WHERE DATE_PART('year', order_date) = 2023) 
"""

In [35]:
result_2 = duckdb.query(query_2).to_df()
display(result_2)

,product_name
0,Падать
1,Адвокат
2,Команда
3,Танцевать
4,Ход
5,Инвалид
6,Бровь
7,Процесс
8,Еврейский
9,Механический


_Пояснение: так как наши данные могут отличаться от тех, что лежат у ревьюера в базе данных, количество товаров, которые ни разу не были куплены в 2023 году, может отличаться. В условии видим, что нужно вывести название лишь одного проукта, тогда как у нас получилось несколько. Возможно, в БД ревьюера такой продукт достоверно один, поэтому формулировка задания именно такая._

### Задание 3:
Выведите сумму всех покупок из Москвы за 2024 год.

In [215]:
query_3 = """
SELECT SUM(quantity * price) AS total_revenue_2024
FROM orders o 
LEFT JOIN products p USING(product_id)
LEFT JOIN buyers b USING(buyer_id)
WHERE DATE_PART('year', order_date) = 2024 AND city = 'Москва'
"""

In [216]:
result_3 = duckdb.query(query_3).to_df()
display(result_3)

,total_revenue_2024
0,1440554.33
